# **ECG TECHNICIAN ASSIGNMENT OPTIMIZATION MODEL**

In [12]:
# pip install pulp

In [16]:
import pandas as pd
import pulp
import numpy as np
from datetime import datetime

In [20]:
# Load the fault dataset
df_faults = pd.read_csv('../dataset/ecg_faults_dataset.csv')
df_faults.head()

,Fault_ID,Town,Zone,Distance_km,Travel_time_hours,Fault_type,Repair_time_hours,Priority
0,F1,Afadzo South,Far,38.4,1.017,Pole replacement,1.00,High
1,F2,Liati,Far,20.0,0.550,Transformer maintenance,0.75,Normal
2,F3,Golokwati,Far,21.3,0.500,Transformer maintenance,0.75,High
3,F4,Agome yo,Near,15.9,0.517,Network line extension,2.00,Normal
4,F5,Logba,Far,38.3,1.000,Network line extension,2.00,Normal


In [15]:
# summary
print("DATA SUMMARY")
print(f"Faults by Zone:")
print(df_faults['Zone'].value_counts())
print(f"\nFaults by Type:")
print(df_faults['Fault_type'].value_counts())
print(f"\nAverage travel time (Near): {df_faults[df_faults['Zone']=='Near']['Travel_time_hours'].mean():.3f} hours")
print(f"Average travel time (Far): {df_faults[df_faults['Zone']=='Far']['Travel_time_hours'].mean():.3f} hours")

DATA SUMMARY
Faults by Zone:
Zone
Near    8
Far     7
Name: count, dtype: int64

Faults by Type:
Fault_type
Transformer maintenance          5
Cable joining and termination    4
Network line extension           3
Transformer installation         2
Pole replacement                 1
Name: count, dtype: int64

Average travel time (Near): 0.469 hours
Average travel time (Far): 0.748 hours


#### **EXTRACTING PARAMETERS FROM FAULTS DATASET**

In [31]:
# Define problem dimensions
num_faults = len(df_faults)
num_groups = 5

# Create index sets
faults = list(range(num_faults))  # set J 
groups = list(range(num_groups))   # set I 

print(f"Sets defined:")
print(f"  I (groups): {groups} = {num_groups} groups")
print(f"  J (faults): {faults[0]} to {faults[-1]} = {num_faults} faults")

Sets defined:
  I (groups): [0, 1, 2, 3, 4] = 5 groups
  J (faults): 0 to 14 = 15 faults


In [25]:
# Extract parameters from data
travel_time = df_faults['Travel_time_hours'].tolist()  # t_j
repair_time = df_faults['Repair_time_hours'].tolist()  # r_j
zones = df_faults['Zone'].tolist()                      # z_j

print(f"\nParameters extracted:")
print(f"  t_j (travel times): {len(travel_time)} values")
print(f"  r_j (repair times): {len(repair_time)} values")
print(f"  z_j (zones): {len(zones)} values")


Parameters extracted:
  t_j (travel times): 15 values
  r_j (repair times): 15 values
  z_j (zones): 15 values


In [27]:
# Define other parameters
Q = 3           # Capacity per group
H = 8           # Shift duration in hours
theta = 1.5     # Equity tolerance (Far can be 1.5x Near)
alpha = 0.7     # Efficiency weight
beta = 0.3      # Equity weight

print(f"\nFixed parameters:")
print(f"  Q (capacity per group): {Q} faults")
print(f"  H (shift duration): {H} hrs")
print(f"  θ (equity tolerance): {theta}")
print(f"  α (efficiency weight): {alpha}")
print(f"  β (equity weight): {beta}")


Fixed parameters:
  Q (capacity per group): 3 faults
  H (shift duration): 8 hrs
  θ (equity tolerance): 1.5
  α (efficiency weight): 0.7
  β (equity weight): 0.3


In [30]:
# Identify zones indexes
near_faults = [j for j in faults if zones[j] == 'Near']
far_faults = [j for j in faults if zones[j] == 'Far']

print(f"\nZones:")
print(f"  Near faults: {near_faults}")
print(f"  Far faults: {far_faults}")


Zones:
  Near faults: [3, 6, 7, 8, 10, 12, 13, 14]
  Far faults: [0, 1, 2, 4, 5, 9, 11]


#### **CREATING OPTIMIZATION MODEL**

In [33]:
# Create the model
model = pulp.LpProblem("ECG_Technician_Assignment", pulp.LpMinimize)
model

ECG_Technician_Assignment:
MINIMIZE
None
VARIABLES

#### Define decision variables

In [35]:
# Decision variables: x[i,j] = 1 if group i assigned to fault j, 0 otherwise
x = pulp.LpVariable.dicts(
    "assign",
    [(i, j) for i in groups for j in faults],
    cat='Binary'
)

print(f"Created {len(x)} binary decision variables x_ij")
print(f"Format: x[i,j] where i ∈ {{{groups[0]},...,{groups[-1]}}} and j ∈ {{{faults[0]},...,{faults[-1]}}}")

Created 75 binary decision variables x_ij
Format: x[i,j] where i ∈ {0,...,4} and j ∈ {0,...,14}


#### Define objective function

In [37]:
# obj function
objective = pulp.lpSum([
    travel_time[j] * x[i, j]
    for i in groups
    for j in faults
])

model += objective, "Total_Travel_Time"

print("Objective function added: Minimize Total Travel Time")
print("Formula: Σ_i Σ_j (t_j × x_ij)")

Objective function added: Minimize Total Travel Time
Formula: Σ_i Σ_j (t_j × x_ij)


/home/kalamix/dsX/lib/python3.12/site-packages/pulp/pulp.py:1865: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")
